# Segmenting and Clustering Neighborhoods in Toronto

Links to parts:
* [part_1](#part_1)
* [part_2](#part_2)
* [part_3](#part_3)

<a id='part_1'></a>

## Part 1 - Extracting Data from Wikipedia and processing it into a Dataframe

#### Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [11]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [12]:
#Import the Libaries that are needed
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup as bs
print("Done")

Done


In [13]:
#Define the URL
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#get the URL and parse the HTML
page = requests.get(url)
soup = bs(page.text, 'lxml')

data = []
columns = []

#Go throught the HTML, find the Table and interate through the table to find the values
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    if (index == 0):
        columns = section
    else:
        data.append(section)

#Convert scraped data to dataframe
df = pd.DataFrame(data = data,columns = columns)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [14]:
#remove any cells in the borough column that have a value of Not assigned
df = df[~df['Borough'].isin(['Not assigned'])]
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [15]:
#group by postcode and merge Neighbourhoods with the same borough and postcode
df["Neighbourhood"] = df.groupby("Postcode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
5,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"


In [16]:
#Drop the duplicate cells
df = df.drop_duplicates()
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Not assigned


In [17]:
df.shape

(103, 3)

<a id='part_2'></a>

# Part 2 - Get GEO Location Data Using GEO Coder

Geocoder didn't work for my work book. I have therefore used the CSV (Geospatial_Coordinates.csv) to import the latitude and longitude data for each Borough.

In [18]:
#Get data from the CSV and write it to a dataframe
file = 'http://cocl.us/Geospatial_data'
df_latlong = pd.read_csv(file)

df_latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df_merge = pd.merge(df, df_latlong, left_on='Postcode', right_on='Postal Code')
del df_merge['Postal Code']

print(df_merge.shape)
df_merge.head()

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494


<a id='part_3'></a>

# Part 3 - Explore and cluster the neighborhoods in Toronto.

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1 - to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2 - to generate maps to visualize your neighborhoods and how they cluster together.

Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

## Explanation about what the part 3 analysis will entail

The goal of this section is to get fimiliar with the lines of code used in the segmentation of New York City data in the modules labs. This will also give some fimilarity with working with Foursquare API in a new regions. The analysis will aim to replicate to the New York analysis for Toronto. I have added additional comments to the top of some cells to explain any changes and why.

In [20]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_merge['Borough'].unique()),
        df_merge.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [21]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [22]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [23]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [60]:
# @hidden_cell
CLIENT_ID = 'DELETED' # your Foursquare ID
CLIENT_SECRET = 'DELETED' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails have been successfully imported')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails have been successfully imported


In [25]:
df_merge.loc[0, 'Neighbourhood']

'Parkwoods'

In [26]:
neighborhood_latitude = df_merge.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_merge.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_merge.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [61]:
#I have increased to radius from 500 to 2,000 to get more results to work with
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 2000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=DELETED&client_secret=DELETED&v=20180605&ll=43.7532586,-79.3296565&radius=2000&limit=100'

In [28]:
#results from Foursquare API call
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5db734f048b1e1002b7c47be'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 93,
  'suggestedBounds': {'ne': {'lat': 43.77125861800002,
    'lng': -79.30478345939711},
   'sw': {'lat': 43.735258581999986, 'lng': -79.35452954060288}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b8991cbf964a520814232e3',
       'name': "Allwyn's Bakery",
       'location': {'address': '81 Underhill drive',
        'lat': 43.75984035203157,
        'lng': -79.32471879917513,
        'labeledLatLngs': [{'label': 'display'

In [29]:
import json # library to handle JSON files
from pandas.io.json import json_normalize
print('Done')

Done


In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,Donalda Golf & Country Club,Golf Course,43.752816,-79.342741
2,LCBO,Liquor Store,43.757774,-79.314257
3,Graydon Hall Manor,Event Space,43.763923,-79.342961
4,Darband Restaurant,Middle Eastern Restaurant,43.755194,-79.348498


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

93 venues were returned by Foursquare.


In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
toronto_venues = getNearbyVenues(names=df_merge['Neighbourhood'],
                                   latitudes=df_merge['Latitude'],
                                   longitudes=df_merge['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Not assigned
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [35]:
print(toronto_venues.shape)
toronto_venues.head()

(8595, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store
3,Parkwoods,43.753259,-79.329656,Graydon Hall Manor,43.763923,-79.342961,Event Space
4,Parkwoods,43.753259,-79.329656,Darband Restaurant,43.755194,-79.348498,Middle Eastern Restaurant


In [36]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,100,100,100,100,100,100
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",98,98,98,98,98,98
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",48,48,48,48,48,48
"Alderwood, Long Branch",100,100,100,100,100,100
"Bathurst Manor, Downsview North, Wilson Heights",56,56,56,56,56,56
Bayview Village,43,43,43,43,43,43
"Bedford Park, Lawrence Manor East",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100


In [37]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 329 uniques categories.


In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Road,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaur

In [39]:
toronto_onehot.shape

(8595, 329)

In [40]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Road,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaur

In [41]:
toronto_grouped.shape

(103, 329)

In [42]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                           venue  freq
0                    Coffee Shop  0.06
1                          Hotel  0.05
2                        Theater  0.04
3  Vegetarian / Vegan Restaurant  0.03
4                      Gastropub  0.03


----Agincourt----
                venue  freq
0  Chinese Restaurant  0.12
1         Coffee Shop  0.09
2            Pharmacy  0.05
3          Restaurant  0.04
4      Sandwich Place  0.03


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0  Chinese Restaurant  0.18
1         Coffee Shop  0.07
2                Park  0.05
3         Pizza Place  0.04
4              Bakery  0.04


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Coffee Shop  0.15
1     Indian Restaurant  0.10
2           Pizza Place  0.10
3  Fast Food Restaurant  0.10
4         Grocery Store  0.06


----Alde

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Hotel,Theater,Vegetarian / Vegan Restaurant,Beer Bar,Cosmetics Shop,Pizza Place,Café,Gastropub,Restaurant
1,Agincourt,Chinese Restaurant,Coffee Shop,Pharmacy,Restaurant,Sandwich Place,Indian Restaurant,Breakfast Spot,Gas Station,Bakery,Bank
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Coffee Shop,Park,Pizza Place,Bakery,Pharmacy,Indian Restaurant,Japanese Restaurant,Noodle House,Bubble Tea Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Coffee Shop,Fast Food Restaurant,Pizza Place,Indian Restaurant,Grocery Store,Sandwich Place,Skating Rink,Park,Discount Store,Café
4,"Alderwood, Long Branch",Coffee Shop,Fast Food Restaurant,Burger Joint,Department Store,Pizza Place,Pharmacy,Restaurant,Breakfast Spot,Café,Electronics Store


In [45]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 3, 0, 3, 3, 3, 3, 1, 3], dtype=int32)

In [46]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Clusters', kmeans.labels_)

neighborhoods_venues_sorted.head()

,Clusters,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Adelaide, King, Richmond",Coffee Shop,Hotel,Theater,Vegetarian / Vegan Restaurant,Beer Bar,Cosmetics Shop,Pizza Place,Café,Gastropub,Restaurant
1,3,Agincourt,Chinese Restaurant,Coffee Shop,Pharmacy,Restaurant,Sandwich Place,Indian Restaurant,Breakfast Spot,Gas Station,Bakery,Bank
2,3,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Coffee Shop,Park,Pizza Place,Bakery,Pharmacy,Indian Restaurant,Japanese Restaurant,Noodle House,Bubble Tea Shop
3,0,"Albion Gardens, Beaumond Heights, Humbergate, ...",Coffee Shop,Fast Food Restaurant,Pizza Place,Indian Restaurant,Grocery Store,Sandwich Place,Skating Rink,Park,Discount Store,Café
4,3,"Alderwood, Long Branch",Coffee Shop,Fast Food Restaurant,Burger Joint,Department Store,Pizza Place,Pharmacy,Restaurant,Breakfast Spot,Café,Electronics Store


In [47]:
df_merge.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494


In [48]:
toronto_merge = pd.merge(neighborhoods_venues_sorted, df_merge, left_on='Neighborhood', right_on='Neighbourhood')
toronto_merge.head()

,Clusters,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,1,"Adelaide, King, Richmond",Coffee Shop,Hotel,Theater,Vegetarian / Vegan Restaurant,Beer Bar,Cosmetics Shop,Pizza Place,Café,Gastropub,Restaurant,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
1,3,Agincourt,Chinese Restaurant,Coffee Shop,Pharmacy,Restaurant,Sandwich Place,Indian Restaurant,Breakfast Spot,Gas Station,Bakery,Bank,M1S,Scarborough,Agincourt,43.794200,-79.262029
2,3,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Coffee Shop,Park,Pizza Place,Bakery,Pharmacy,Indian Restaurant,Japanese Restaurant,Noodle House,Bubble Tea Shop,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577
3,0,"Albion Gardens, Beaumond Heights, Humbergate, ...",Coffee Shop,Fast Food Restaurant,Pizza Place,Indian Restaurant,Grocery Store,Sandwich Place,Skating Rink,Park,Discount Store,Café,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
4,3,"Alderwood, Long Branch",Coffee Shop,Fast Food Restaurant,Burger Joint,Department Store,Pizza Place,Pharmacy,Restaurant,Breakfast Spot,Café,Electronics Store,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484


In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merge['Latitude'], toronto_merge['Longitude'], toronto_merge['Neighborhood'], toronto_merge['Clusters']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
toronto_merge.loc[toronto_merge['Clusters'] == 0, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Neighborhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode,Borough,Neighbourhood,Latitude,Longitude
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Indian Restaurant,Grocery Store,Sandwich Place,Skating Rink,Park,Discount Store,Café,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
23,"Clairlea, Golden Mile, Oakridge",Burger Joint,Pizza Place,Grocery Store,Chinese Restaurant,Burrito Place,Bakery,Intersection,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
24,"Clarks Corners, Sullivan, Tam O'Shanter",Coffee Shop,Pharmacy,Chinese Restaurant,Sandwich Place,Seafood Restaurant,Falafel Restaurant,Restaurant,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302
25,"Cliffcrest, Cliffside, Scarborough Village West",Coffee Shop,Beach,Pharmacy,Sandwich Place,Grocery Store,Pizza Place,Cajun / Creole Restaurant,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
34,"Dorset Park, Scarborough Town Centre, Wexford ...",Pizza Place,Pharmacy,Breakfast Spot,Pet Store,Park,Grocery Store,Wings Joint,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304
36,Downsview Central,Pharmacy,Sandwich Place,Grocery Store,Bank,Pizza Place,Convenience Store,Supermarket,M3M,North York,Downsview Central,43.728496,-79.495697
37,Downsview Northwest,Pizza Place,Grocery Store,Sandwich Place,Tennis Stadium,Pharmacy,Snack Place,Event Service,M3N,North York,Downsview Northwest,43.761631,-79.520999
38,Downsview West,Vietnamese Restaurant,Spa,Coffee Shop,Fast Food Restaurant,Intersection,Bank,Tea Room,M3L,North York,Downsview West,43.739015,-79.506944
39,"Downsview, North Park, Upwood Park",Supermarket,Pizza Place,Grocery Store,Sandwich Place,Bank,Discount Store,Pharmacy,M6L,North York,"Downsview, North Park, Upwood Park",43.713756,-79.490074
40,"East Birchmount Park, Ionview, Kennedy Park",Coffee Shop,Pharmacy,Beer Store,Bank,Sandwich Place,Discount Store,Fried Chicken Joint,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029


In [51]:
toronto_merge.loc[toronto_merge['Clusters'] == 1, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Neighborhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,"Adelaide, King, Richmond",Vegetarian / Vegan Restaurant,Beer Bar,Cosmetics Shop,Pizza Place,Café,Gastropub,Restaurant,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,Berczy Park,Park,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,Farmers Market,Beer Bar,Gastropub,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
14,"CN Tower, Bathurst Quay, Island airport, Harbo...",Hotel,Brewery,Gym,Italian Restaurant,Dessert Shop,Beer Bar,Seafood Restaurant,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
19,Central Bay Street,Restaurant,Park,Cosmetics Shop,Theater,Japanese Restaurant,Plaza,Sushi Restaurant,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
27,"Commerce Court, Victoria Hotel",Café,Plaza,Seafood Restaurant,Park,Gastropub,Cosmetics Shop,Concert Hall,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817
32,"Design Exchange, Toronto Dominion Centre",Park,Gastropub,Japanese Restaurant,Seafood Restaurant,Café,Restaurant,Baseball Stadium,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
44,"First Canadian Place, Underground city",Theater,Park,Gastropub,Café,Plaza,Pizza Place,Beer Bar,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
50,"Harbourfront East, Toronto Islands, Union Station",Park,Beer Bar,Italian Restaurant,Gym,Theater,Pizza Place,Dessert Shop,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
51,"Harbourfront, Regent Park",Bakery,Restaurant,Japanese Restaurant,Italian Restaurant,Gastropub,Farmers Market,Thai Restaurant,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
73,Not assigned,Mexican Restaurant,Park,Cosmetics Shop,Pizza Place,Hotel,Arts & Crafts Store,Comic Shop,M7A,Queen's Park,Not assigned,43.662301,-79.389494


In [52]:
toronto_merge.loc[toronto_merge['Clusters'] == 2, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Neighborhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode,Borough,Neighbourhood,Latitude,Longitude
11,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Bakery,Asian Restaurant,Seafood Restaurant,Hotel,Tea Room,Athletics & Sports,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
12,Business Reply Mail Processing Centre 969 Eastern,Brewery,Italian Restaurant,Pizza Place,Beach,Indian Restaurant,American Restaurant,Bakery,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
15,"Cabbagetown, St. James Town",Restaurant,Diner,Gastropub,Thai Restaurant,Japanese Restaurant,Italian Restaurant,Dance Studio,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
16,Caledonia-Fairbanks,Sandwich Place,Mexican Restaurant,Furniture / Home Store,Bank,Pizza Place,Burger Joint,Breakfast Spot,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
20,"Chinatown, Grange Park, Kensington Market",Bar,Coffee Shop,Sandwich Place,Art Gallery,Dessert Shop,Beer Bar,French Restaurant,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049
21,Christie,Korean Restaurant,Grocery Store,Indian Restaurant,Italian Restaurant,Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,M6G,Downtown Toronto,Christie,43.669542,-79.422564
22,Church and Wellesley,Japanese Restaurant,Park,Pizza Place,Hotel,Ramen Restaurant,Restaurant,Gay Bar,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
28,Davisville,Park,Bakery,Gym,Japanese Restaurant,Indian Restaurant,Yoga Studio,Gastropub,M4S,Central Toronto,Davisville,43.704324,-79.388790
29,Davisville North,Park,Bakery,Gym,Sporting Goods Shop,Indian Restaurant,Japanese Restaurant,Sushi Restaurant,M4P,Central Toronto,Davisville North,43.712751,-79.390197
30,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Coffee Shop,Grocery Store,American Restaurant,Spa,Ice Cream Shop,Hotel,French Restaurant,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [53]:
toronto_merge.loc[toronto_merge['Clusters'] == 3, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Neighborhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode,Borough,Neighbourhood,Latitude,Longitude
1,Agincourt,Restaurant,Sandwich Place,Indian Restaurant,Breakfast Spot,Gas Station,Bakery,Bank,M1S,Scarborough,Agincourt,43.794200,-79.262029
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Pizza Place,Bakery,Pharmacy,Indian Restaurant,Japanese Restaurant,Noodle House,Bubble Tea Shop,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577
4,"Alderwood, Long Branch",Department Store,Pizza Place,Pharmacy,Restaurant,Breakfast Spot,Café,Electronics Store,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
5,"Bathurst Manor, Downsview North, Wilson Heights",Park,Fast Food Restaurant,Restaurant,Bank,Convenience Store,Pharmacy,Deli / Bodega,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259
6,Bayview Village,Park,Pharmacy,Café,Clothing Store,Grocery Store,Trail,Taiwanese Restaurant,M2K,North York,Bayview Village,43.786947,-79.385975
7,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Bagel Shop,Restaurant,Pub,Bank,Pizza Place,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
9,"Birch Cliff, Cliffside West",Pharmacy,Bank,Beer Store,Fast Food Restaurant,Bakery,Bar,Sushi Restaurant,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
10,"Bloordale Gardens, Eringate, Markland Wood, Ol...",Park,Golf Course,Grocery Store,Garden,Gym,Beer Store,Baseball Field,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
13,"CFB Toronto, Downsview East",Cosmetics Shop,Furniture / Home Store,Vietnamese Restaurant,American Restaurant,Pizza Place,Fast Food Restaurant,Dessert Shop,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763
17,Canada Post Gateway Processing Centre,Sandwich Place,Japanese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Mexican Restaurant,Restaurant,Trail,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819


In [54]:
toronto_merge.loc[toronto_merge['Clusters'] == 4, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Neighborhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode,Borough,Neighbourhood,Latitude,Longitude
78,"Rouge, Malvern",Athletics & Sports,Zoo,Spa,Grocery Store,Fried Chicken Joint,Sandwich Place,Fruit & Vegetable Store,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353


The clusters show varied results but a commonality across the clusters appears to be that Coffee shops are very popular in Neighborhood's in Toronto. If a further analysis was done on this data I would exclude coffee shops as a venue as it maybe interesting to see an analysis without this.